In [24]:
import timm
import torch
from torch.utils.data import Dataset
import torchvision
from tqdm import tqdm
import torchvision.transforms as T
import torchvision.transforms.functional as TF

torch.set_grad_enabled(False)
device = torch.device("cuda:0")
every_n_frames = 1

In [14]:
model_name = "dummy"

class DummyModel:
    def forward_features(self, x):
        return torch.rand(x.size(0), 197, 768)

In [18]:
model_name = "crocov2"

%cd /root/croco
from models.croco import CroCoNet
ckpt = torch.load('CroCo_V2_ViTBase_BaseDecoder.pth', 'cpu')
model = CroCoNet( **ckpt.get('croco_kwargs',{}))
msg = model.load_state_dict(ckpt['model'], strict=True)
use_gpu = torch.cuda.is_available() and torch.cuda.device_count()>0
device = torch.device('cuda:0' if use_gpu else 'cpu')
model = model.eval()
model = model.to(device=device)
print(msg)

/root/croco
Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


/tmp/ipykernel_2527/962728620.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('CroCo_V2_ViTBase_BaseDecoder.pth', 'cpu')


<All keys matched successfully>


In [15]:
model_name="spa"

state_dict = torch.load("spa-b.ckpt")
state_dict = state_dict["state_dict"]

for k in list(state_dict.keys()):
    if not k.startswith("model.img_backbone"):
        del state_dict[k]
    else:
        new_key = k.replace("model.img_backbone.", "")
        state_dict[new_key] = state_dict.pop(k)

model = timm.create_model('vit_base_patch16_224', pretrained=False).cuda()
model.head = torch.nn.Identity()
model.fc_norm = torch.nn.Identity()

model.load_state_dict(state_dict, strict=False)

/tmp/ipykernel_2527/1235756427.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("spa-b.ckpt")


_IncompatibleKeys(missing_keys=[], unexpected_keys=['mask_token', 'decoder_embed.weight', 'decoder_embed.bias', 'readout_project.0.weight', 'readout_project.0.bias', 'upsample.0.conv.weight', 'upsample.1.conv.weight'])

In [28]:
model_name="dino"

model = timm.create_model(f'vit_base_patch16_224.{model_name}', pretrained=True).cuda()
model.head = torch.nn.Identity()
model.fc_norm = torch.nn.Identity()

In [29]:
dataset = torchvision.datasets.ImageFolder(
    f"frames_{every_n_frames}", 
    transform=T.Compose([
        T.Resize(224),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
    ])
)
    
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

In [30]:

all_features = {cls:[] for cls in dataset.classes}

for img, label in tqdm(dataloader):
    features = model.forward_features(img.to(device))

    for f, l in zip(features, label):
        name = dataset.classes[l]
        all_features[name].append(f)

for cls, features in all_features.items():
    all_features[cls] = torch.stack(features).cpu()

torch.save(all_features, f"repr_{every_n_frames}/repr_{model_name}.pt")

/root/new


100%|██████████| 299/299 [10:57<00:00,  2.20s/it]
